In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/houseplants.zip"  # Zip dosyanın yolu
extract_to = "/content/houseplants"  # Hedef klasör

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ houseplants.zip başarıyla çıkarıldı.")

✅ houseplants.zip başarıyla çıkarıldı.


In [3]:
!ls /content/houseplants

house_plant_species


In [4]:
!pip install -q torch torchvision ftfy regex tqdm
!pip install -q git+https://github.com/mlfoundations/open_clip.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Ge

In [5]:
import torch
import open_clip

device = "cuda" if torch.cuda.is_available() else "cpu"

model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="ViT-B-32",  # veya ViT-L-14 ya da ViT-H-14 kullanabilirsin
    pretrained="laion2b_s34b_b79k"
)
model = model.to(device).eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [6]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

# Klasör yolu
image_dir = "/content/houseplants/house_plant_species"

# ImageFolder datasetini hazırla
dataset = ImageFolder(root=image_dir, transform=preprocess)

# Veri yükleyici (batch size küçük tutabiliriz, örn: 8)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

In [7]:
import numpy as np
from tqdm import tqdm

all_embeddings = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(dataloader):
        images = images.to(device)
        embeddings = model.encode_image(images)
        embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)  # normalize et
        all_embeddings.append(embeddings.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_embeddings = np.vstack(all_embeddings)
all_labels = np.hstack(all_labels)

  7%|▋         | 131/1847 [04:18<51:18,  1.79s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 1847/1847 [1:00:17<00:00,  1.96s/it]


In [8]:
class_names = dataset.classes
print(class_names)

['African Violet (Saintpaulia ionantha)', 'Aloe Vera', 'Anthurium (Anthurium andraeanum)', 'Areca Palm (Dypsis lutescens)', 'Asparagus Fern (Asparagus setaceus)', 'Begonia (Begonia spp.)', 'Bird of Paradise (Strelitzia reginae)', 'Birds Nest Fern (Asplenium nidus)', 'Boston Fern (Nephrolepis exaltata)', 'Calathea', 'Cast Iron Plant (Aspidistra elatior)', 'Chinese Money Plant (Pilea peperomioides)', 'Chinese evergreen (Aglaonema)', 'Christmas Cactus (Schlumbergera bridgesii)', 'Chrysanthemum', 'Ctenanthe', 'Daffodils (Narcissus spp.)', 'Dracaena', 'Dumb Cane (Dieffenbachia spp.)', 'Elephant Ear (Alocasia spp.)', 'English Ivy (Hedera helix)', 'Hyacinth (Hyacinthus orientalis)', 'Iron Cross begonia (Begonia masoniana)', 'Jade plant (Crassula ovata)', 'Kalanchoe', 'Lilium (Hemerocallis)', 'Lily of the valley (Convallaria majalis)', 'Money Tree (Pachira aquatica)', 'Monstera Deliciosa (Monstera deliciosa)', 'Orchid', 'Parlor Palm (Chamaedorea elegans)', 'Peace lily', 'Poinsettia (Euphorbia 

In [9]:
from google.colab import files
uploaded = files.upload()

Saving şeflera.jpg to şeflera.jpg


In [10]:
from PIL import Image
import torchvision.transforms as transforms

# Test edilecek görselin yolu
image_path = "şeflera.jpg"  # senin dışarıdan yüklediğin fotoğraf

# Görseli oku ve modele uygun boyuta getir
image = Image.open(image_path).convert("RGB")

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                         std=[0.26862954, 0.26130258, 0.27577711]),
])

image_tensor = preprocess(image).unsqueeze(0).to(device)


In [11]:
with torch.no_grad():
    query_embedding = model.encode_image(image_tensor)
    query_embedding = query_embedding / query_embedding.norm(dim=-1, keepdim=True)

In [12]:
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

unique_labels = np.unique(all_labels)
class_embeddings = []

for label in unique_labels:
    class_embeds = all_embeddings[all_labels == label]
    class_mean = class_embeds.mean(axis=0)
    class_embeddings.append(class_mean)

class_embeddings = torch.tensor(class_embeddings).to(device)

/tmp/ipython-input-1854931365.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  class_embeddings = torch.tensor(class_embeddings).to(device)


In [13]:
print("Class embeddings shape:", class_embeddings.shape)
print("Number of classes:", len(class_names))

Class embeddings shape: torch.Size([47, 512])
Number of classes: 47


In [14]:
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.48145466, 0.4578275, 0.40821073],
        std=[0.26862954, 0.26130258, 0.27577711]
    )
])

test_image_path = "şeflera.jpg"  # kendi test resmin buraya
image = Image.open(test_image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    test_embedding = model.encode_image(image_tensor)
    test_embedding = test_embedding / test_embedding.norm(dim=-1, keepdim=True)

similarities = F.cosine_similarity(test_embedding, class_embeddings)
predicted_index = similarities.argmax().item()
predicted_class = class_names[predicted_index]

print(f"Tahmin edilen sınıf: {predicted_class}")


Tahmin edilen sınıf: Schefflera


In [15]:
from google.colab import files
uploaded = files.upload()

Saving Adsız doküman.txt to Adsız doküman.txt


In [16]:
def parse_plant_care_final(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()

    entries = text.split("🌿")[1:]  # her bitki bloğu
    plant_dict = {}

    for entry in entries:
        lines = entry.strip().splitlines()
        if not lines:
            continue

        title_line = lines[0].strip()

        # Sadece bitki adını çek: parantez öncesi ve "Bakımı" temizlenir
        name = title_line.split("(")[0].replace("Bakımı", "").strip().lower()

        content = "\n".join(lines[1:]).strip()

        if name not in plant_dict:
            plant_dict[name] = content

    return plant_dict


In [17]:
plant_dict = parse_plant_care_final("/content/Adsız doküman.txt")

In [18]:
print("şeflera" in plant_dict)  # ✅ True dönmeli

True


In [21]:
def find_closest_plant_name(predicted_class, plant_dict_keys):
    """Finds the closest plant name in the dictionary keys using difflib."""
    closest_match = difflib.get_close_matches(predicted_class.lower(), plant_dict_keys, n=1, cutoff=0.6)
    if closest_match:
        return closest_match[0]
    return None

def show_care_info_v2(predicted_class, plant_dict):
    """
    Retrieves and displays care information for a predicted plant class,
    using fuzzy matching to find the closest key in the plant_dict.
    """
    # Normalize predicted class name for better matching
    normalized_predicted_class = predicted_class.split("(")[0].strip().lower()

    # Find the closest matching key in the plant_dict
    closest_key = find_closest_plant_name(normalized_predicted_class, plant_dict.keys())

    if closest_key and closest_key in plant_dict:
        print(f"🌿 Bakım Önerisi ({predicted_class}):")
        print(plant_dict[closest_key])
    else:
        print(f"❌ Bakım bilgisi bulunamadı: {predicted_class}")

# 🌿 Bakım önerisini göster
show_care_info_v2(predicted_class, plant_dict)

🌿 Bakım Önerisi (Schefflera):
Genel Bakım:
                                                                                                                                                                                                                                                                                                                                       * Işık:
 Parlak ancak dolaylı ışığı sever. Doğrudan güneş ışığı yapraklarını yakabilir.
(Prefers bright but indirect light. Direct sunlight can scorch its leaves.)

                                                                                                                                                                                                                                                                                                                                       * Sulama:
 Toprağın üst kısmı kurudukça sulanmalıdır. Aşırı sulama kök çürümesine neden olabilir.
(Water when the topsoil dries out. Overw